# Variational Quantum Error Correction with QVECTOR

In this notebook, we will optimize a Quantum Error Correction (QEC) problem. 
Here, our set of states $\{\psi_i\}_{i=1}^N$ are sampled from the Haar distribution (or an approximation thereof) and we optimize the cost 
$$L(\vec{p}, \vec{q}) = \sum_{\vert \psi_i \rangle \in \mathcal{S}} \frac{1}{\vert\mathcal{S}\vert} E_{i}(\vert \psi_i \rangle, \vec{p}, \vec{q})$$
where $\vec{p}, \vec{q}$ are parameters of a variational circuit, and $E_{i}(\vert \psi_i \rangle, \vec{p}, \vec{q}) = \langle \psi_i \vert \mathcal{V}(\vec{p})^\dagger \mathcal{W}(\vec{q}) \mathcal{N} \mathcal{V}(\vec{p}) \vert \psi_i \rangle$, with $\mathcal{V}$ the encoding operator, $\mathcal{N}$ the noise model, and $\mathcal{W}$ the decoding operator. The noise model here will be a bitflip channel (with probability $0.1$) acting on all qubits.

In [1]:
import typing as ty
import pennylane as qml
from pennylane import numpy as np
from refoqus import Refoqus

# Initialise the number of qubits that interest us.
k, n, r = 1, 5, 0
nbqbits = n + r

In [2]:
layers = 1

def random_angles(*shape: int) -> np.ndarray:
    return 2 * np.pi * np.random.rand(*shape)

if k == 1:
    # Only consider the 6 one qubit stabilizer states:
    dataset_size = 6
    dataset = [
        [qml.Identity(wires=[0])], [qml.PauliX(wires=[0])],  # |0>, |1>
        [qml.Hadamard(wires=[0])], [qml.PauliX(wires=[0]), qml.Hadamard(wires=[0])],  # |+>, |->
        [qml.Hadamard(wires=[0]), qml.S(wires=[0])], [qml.PauliX(wires=[0]), qml.Hadamard(wires=[0]), qml.S(wires=[0])]  # |+i>, |-i>
    ]
    
elif k > 1:
    # Build the dataset here with qml.SimplifiedTwoDesign
    # See https://docs.pennylane.ai/en/stable/code/api/pennylane.SimplifiedTwoDesign.html
    dataset_size = 200
    dataset = [[qml.SimplifiedTwoDesign(initial_layer_weights=random_angles(k), weights=random_angles(layers, k - 1, 2), wires=range(k))] for _ in range(dataset_size)]

In [3]:
# Get the Hamiltonian of interest
coefficients_cost = [- 1.0 / k for _ in range(k)]
projector = np.zeros((2, 2))
projector[0, 0] = 1
vqec_hamiltonian_term = [qml.Hermitian(projector, wires=i) for i in range(k)]
hamiltonian_of_interest = qml.Hamiltonian(coefficients_cost, vqec_hamiltonian_term)

Next, we define functions to evaluate the true cost during optimization.

In [4]:
def circuit_construction(
    weights: np.ndarray,
    data_circuit: ty.List[qml.operation.Operation],
    parameterised_circuit = None,
    noise_circuit = None,
):
    if parameterised_circuit is None:
        parameterised_circuit = qml.StronglyEntanglingLayers
    if noise_circuit is None:
        def noise_circuit():
            for i in range(n):
                qml.BitFlip(0.1, wires=i)

    encoding_shape = parameterised_circuit.shape(layers, n)
    decoding_shape = parameterised_circuit.shape(layers, n + r)
    encoding_size = np.prod(encoding_shape)
    decoding_size = np.prod(decoding_shape)
    encoding_weights = weights[:encoding_size].reshape(encoding_shape)
    decoding_weights = weights[encoding_size:encoding_size + decoding_size].reshape(decoding_shape)
    
    # Apply S         to range(k)     to prepare an approximate 2-design state
    for op in data_circuit:
        qml.apply(op)
    
    # Apply V(p)      to range(n)     to encode the state
    parameterised_circuit(encoding_weights, wires=range(n))
    # Apply potential noise
    noise_circuit()
    # Apply W(q)      to range(n + r) to correct potential errors
    parameterised_circuit(decoding_weights, wires=range(n + r))
    # Apply V^{-1}(p) to range(n)     to decode the state
    qml.adjoint(parameterised_circuit(encoding_weights, wires=range(n)))
    
    # Apply S^{-1}    to range(k)     to un-prepare.
    for op in reversed(data_circuit):
        qml.apply(qml.adjoint(op))
    
def cost_function(
    weights: np.ndarray,
    hamiltonian_terms: qml.operation.Operator,
    data_circuit: ty.List[qml.operation.Operation],
    parameterised_circuit = None,
):
    circuit_construction(weights, data_circuit, parameterised_circuit)
    return qml.sample(hamiltonian_terms)

In [5]:
analytic_dev = qml.device("default.mixed", wires=nbqbits, shots=None)

@qml.qnode(analytic_dev)
def cost_analytic_one_circuit(weights, index_datapoint, parameterised_circuit = None):
    circuit_construction(weights, dataset[index_datapoint], parameterised_circuit, noise_circuit=lambda : None)
    return qml.expval(hamiltonian_of_interest)

def cost_analytic_alldataset(weights, parameterised_circuit = None):
    cost = 0.0
    for m in range(dataset_size):
        cost += cost_analytic_one_circuit(weights, m, parameterised_circuit)
    cost = 1.0 + cost / dataset_size
    return cost

In [6]:
ansatz = qml.StronglyEntanglingLayers
parameter_size: int = np.prod(ansatz.shape(layers, n)) + np.prod(ansatz.shape(layers, n + r))
params = random_angles(parameter_size)
print(qml.draw(cost_analytic_one_circuit, max_length=300)(params, 0))

0: ─╭StronglyEntanglingLayers(M0)─╭StronglyEntanglingLayers(M1)─╭StronglyEntanglingLayers(M0)†──I†──I†─┤  <𝓗(-1.00)>
1: ─├StronglyEntanglingLayers(M0)─├StronglyEntanglingLayers(M1)─├StronglyEntanglingLayers(M0)†─────────┤            
2: ─├StronglyEntanglingLayers(M0)─├StronglyEntanglingLayers(M1)─├StronglyEntanglingLayers(M0)†─────────┤            
3: ─├StronglyEntanglingLayers(M0)─├StronglyEntanglingLayers(M1)─├StronglyEntanglingLayers(M0)†─────────┤            
4: ─╰StronglyEntanglingLayers(M0)─╰StronglyEntanglingLayers(M1)─╰StronglyEntanglingLayers(M0)†─────────┤            


Now, the ansatz is defined as with StronglyEntanglingLayers. We also sample initial values and the corresponding cost.

Our adaptative optimizer will be Refoqus where we provide the necessary arguments as follows and we perform niter iterations.

In [7]:
ansatz = qml.StronglyEntanglingLayers

parameter_size: int = np.prod(ansatz.shape(layers, n)) + np.prod(ansatz.shape(layers, n + r))

opt = Refoqus(nbqbits, dataset, vqec_hamiltonian_term, coefficients_cost, param_shape=(parameter_size,), function_cost_term_tosample=cost_function, min_shots=2, device_name="default.mixed")
params = random_angles(parameter_size)
niter = 20

cost_refoqus = [cost_analytic_alldataset(params, ansatz)]
shots_refoqus = [0]

for i in range(niter):
    params = opt.step(params)
    cost_refoqus.append(cost_analytic_alldataset(params, ansatz))
    shots_refoqus.append(opt.shots_used)
    print(f"Step {i}: cost = {cost_refoqus[-1]}, shots_used = {shots_refoqus[-1]}")

Step 0: cost = 0.46260553303474905, shots_used = 120
Step 1: cost = 0.41162373526179297, shots_used = 300
Step 2: cost = 0.38912858042806997, shots_used = 568
Step 3: cost = 0.3740940481631211, shots_used = 868
Step 4: cost = 0.30817856488618034, shots_used = 1520
Step 5: cost = 0.23512439363049464, shots_used = 2530
Step 6: cost = 0.13878604806345252, shots_used = 4058
Step 7: cost = 0.09376213087180651, shots_used = 5402
Step 8: cost = 0.028434255964407407, shots_used = 8368
Step 9: cost = 0.015615115325303797, shots_used = 10248
Step 10: cost = 0.011156320826706256, shots_used = 13106
Step 11: cost = 0.005518326600643442, shots_used = 19756
Step 12: cost = 0.005095968194110401, shots_used = 28298
Step 13: cost = 0.004274563816907384, shots_used = 40924
Step 14: cost = 0.0031661579764987735, shots_used = 51718
Step 15: cost = 0.0015424174268638957, shots_used = 69758
Step 16: cost = 0.0009048210968317649, shots_used = 101760
Step 17: cost = 0.00035035456496279593, shots_used = 140674